In [ ]:
import numpy as np
from PIL import Image
from GombNet.networks import *
from GombNet.loss_func import GombinatorialLoss, DiceLoss
from GombNet.utils import *
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu
%matplotlib ipympl

import sys
sys.path.append('/Users/austin/Documents/GitHub/DataGenSTEM/DataGenSTEM')
import data_generator as dg

from scipy.ndimage import gaussian_filter, center_of_mass
from skimage.feature import blob_log

In [ ]:
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 12 
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [ ]:
shared_folder = '/Users/austin/Desktop/Gomb-Net aux files'

In [ ]:
gomb_loss_history = np.load(shared_folder + '/Pretrained_models/TwoLeggedGraphene256.pthloss_history.npz')
gomb_train_loss = gomb_loss_history['train_loss_history']
gomb_val_loss = gomb_loss_history['val_loss_history']

unet_loss_history = np.load(shared_folder + '/Pretrained_models/BasicUNet256_nodrop.pthloss_history.npz')
unet_train_loss = unet_loss_history['train_loss_history']
unet_val_loss = unet_loss_history['val_loss_history']

unet_gomb_loss_history = np.load(shared_folder + '/Pretrained_models/BasicUNet256.pthloss_history.npz')
unet_gomb_train_loss = unet_gomb_loss_history['train_loss_history']
unet_gomb_val_loss = unet_gomb_loss_history['val_loss_history']

fig, axs = plt.subplots(3, 1, figsize=(5, 10))

axs[0].plot(unet_train_loss, label='train', color = '#1f77b4')
axs[0].plot(unet_val_loss, label='val', color = '#d62728')
axs[0].set_xlim(0, 29)

axs[1].plot(unet_gomb_train_loss, label='train', color = '#1f77b4')
axs[1].plot(unet_gomb_val_loss, label='val', color = '#d62728')
axs[1].set_xlim(0, 99)

axs[2].plot(gomb_train_loss, label='training', color = '#1f77b4')
axs[2].plot(gomb_val_loss, label='validation', color = '#d62728')
axs[2].set_xlim(0, 29)

for ax in axs:
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.legend(loc='upper right')

fig.tight_layout()
plt.savefig('loss_history', dpi=300)

In [ ]:
# Create dataloaders
images_dir = str(shared_folder + '/Graphene_dataset/images')
labels_dir = str(shared_folder + '/Graphene_dataset/labels')
train_loader, val_loader, test_loader = get_dataloaders(images_dir, labels_dir, batch_size = 1, val_split=0.2, test_split=0.1, seed = 42)

In [ ]:
0, 4

In [ ]:
test_iter = 4 # this is a nice one
test = test_loader.dataset[test_iter][0].unsqueeze(0)
gt = test_loader.dataset[test_iter][1]

fig, ax = plt.subplots(1, 3, figsize=(10, 5))
ax[0].imshow(test[0, 0], cmap='gray')
ax[0].set_title('Input')

titles = ['L1: C', 'L2: C']
for i in range(2):
    ax[i+1].imshow(gt[i].cpu().numpy(), cmap='gray')
    ax[i+1].set_title(titles[i])
for a in ax:
    a.axis('off')
fig.tight_layout()

In [ ]:
input_channels = 1
num_classes = 2
num_filters = [32, 64, 128, 256]
threshold = 0
# test = gaussian_filter(test, sigma=1)
test -= test.min()
test /= test.max()
test = torch.tensor(test).float()

predictions = []
# Unet w Dice Loss
path = str(shared_folder + '/Pretrained_models/BasicUNet256_nodrop.pth')

model = Unet(input_channels, num_classes, num_filters, dropout = 0.2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss = DiceLoss(alpha=2)
checkpoint = torch.load(path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
model.eval()

with torch.no_grad():
    pred = model(test)
    pred = pred.squeeze(0)
    pred = pred.cpu().numpy()
    # pred = (pred > threshold)
    predictions.append(pred)


# Unet w Gomb Loss
path = str(shared_folder + '/Pretrained_models/BasicUNet256.pth')

model = Unet(input_channels, num_classes, num_filters, dropout = 0.2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss = GombinatorialLoss(group_size = num_classes//2, loss = 'Dice', epsilon=1e-6, class_weights = None, alpha=2)
checkpoint = torch.load(path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
model.eval()

with torch.no_grad():
    pred = model(test)
    pred = pred.squeeze(0)
    pred = pred.cpu().numpy()
    # pred = (pred > threshold)
    predictions.append(pred)


# GombNet
path = str(shared_folder + '/Pretrained_models/TwoLeggedGraphene256.pth')

model = TwoLeggedUnet(input_channels, num_classes, num_filters, dropout = 0.2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss = GombinatorialLoss(group_size = num_classes//2, loss = 'Dice', epsilon=1e-6, class_weights = None, alpha=2)
checkpoint = torch.load(path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
model.eval()

with torch.no_grad():
    pred = model(test)
    pred = pred.squeeze(0)
    pred = pred.cpu().numpy()
    # pred = (pred > threshold)
    predictions.append(pred)



In [ ]:
names = ['UNet w Dice Loss', 'UNet w Gomb Loss', 'GombNet']
colors = ['viridis', 'plasma']
for idx, p in enumerate(predictions):
    fig.suptitle(names[idx])
    fig, ax = plt.subplots(1, 3, figsize=(10, 5))
    ax[0].imshow(test[0, 0].cpu().numpy(), cmap='gray')
    ax[0].set_title('Input')

    for i in range(2):
        ax[i+1].imshow(p[i], cmap=colors[i])
    
    for a in ax:
        a.axis('off')
    fig.tight_layout()


In [ ]:
# try a regular blob finder for comparison
blob_im = test.squeeze().cpu().numpy()

blobs = blob_log(blob_im, min_sigma=1, max_sigma=20, num_sigma=5, threshold=0.1)
blobs_com = [center_of_mass(blob_im, blobs[i, 0], blobs[i, 1]) for i in range(blobs.shape[0])]

plt.figure()
plt.imshow(blob_im, cmap='gray')
plt.scatter(blobs[:, 1], blobs[:, 0], c='r', s=20)
plt.axis('off')

In [ ]:
for idx, pred in enumerate(predictions):
    fig, ax = plt.subplots(1, 3, figsize=(20, 8))
    fig.suptitle(names[idx])
    ax[0].imshow(test[0, 0].cpu().numpy(), cmap='gray')
    ax[0].set_title('Input')

    colors = ['r', 'g']
    for i in range(2):
        ax[i+1].imshow(blob_im, cmap='gray', vmin=-1,vmax=0)
        blobs = blob_log(pred[i], min_sigma=3, max_sigma=20, num_sigma=10, threshold=0.2)
        blobs_co1 = [center_of_mass(pred[i,0], blobs[j, 0], blobs[j, 1]) for j in range(blobs.shape[0])]
        ax[1].scatter(blobs[:, 1], blobs[:, 0], c=colors[i], s=80, alpha = 0.8, edgecolors='k', linewidth=0.5)

    for a in ax:
        a.axis('off')
        a.set_ylim(0, 256)
        a.set_xlim(0, 256)
    fig.tight_layout()

### Bar plot below

In [ ]:
def iou(pred, gt):
    intersection = np.logical_and(pred, gt).sum()
    union = np.logical_or(pred, gt).sum()
    return intersection / union

def dice_coefficient(pred, gt):
    intersection = np.logical_and(pred, gt).sum()
    return 2 * intersection / (pred.sum() + gt.sum())

# Function to count the number of trainable parameters
def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [ ]:
# Random guessing benchmark
pwa_total = 0
dice_total = 0
IoU_total = 0

for i in range(len(test_loader)):
    gt = test_loader.dataset[i][1].numpy()  # Convert to numpy array
    
    # Random guessing
    prediction = np.random.randint(0, 2, gt.shape)

    pwa_original = np.sum(prediction == gt) / np.prod(gt.shape)
    dice_original = dice_coefficient(prediction, gt)
    iou_original = iou(prediction, gt)
    
    pwa_total += pwa_original
    dice_total += dice_original
    IoU_total += iou_original

pwa_total /= len(test_loader)
dice_total /= len(test_loader)
IoU_total /= len(test_loader)

acc_random = [pwa_total, dice_total, IoU_total]
print("Random")
print(f"Pixel-wise Accuracy: {pwa_total}")
print(f"Mean Dice Coefficient: {dice_total}")
print(f"Mean IoU: {IoU_total}")

In [ ]:
# Regular Unet Benchmark
input_channels = 1
num_classes = 2
num_filters = [32, 64, 128, 256]

model = Unet(input_channels, num_classes, num_filters, dropout = 0.2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss = DiceLoss(alpha=2)

model_path = '/Users/austin/Documents/GitHub/GombNet/trained_models/BasicUNet256_nodrop.pth'
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
model.eval()

pwa_total = 0
dice_total = 0
IoU_total = 0

# Calculate the accuracy
for i in range(len(test_loader)):
    test = test_loader.dataset[i][0].unsqueeze(0)
    gt = test_loader.dataset[i][1].numpy()  # Convert to numpy array
    
    with torch.no_grad():
        probability = model(test)
        prediction = torch.sigmoid(probability)  # Use torch.sigmoid instead of F.sigmoid (deprecated)
    
    probability = probability.squeeze().cpu().numpy()
    prediction = prediction.squeeze().cpu().numpy()

    threshold = threshold_otsu(prediction)
    prediction = (prediction > threshold).astype(float)
    
    # Calculate metrics for original and switched ground truths
    pwa_original = np.sum(prediction == gt) / np.prod(gt.shape)
    dice_original = dice_coefficient(prediction, gt)
    iou_original = iou(prediction, gt)
    
    # There is not combinatorial here
    pwa_total += pwa_original
    dice_total += dice_original
    IoU_total += iou_original

# Calculate the average for each metric
pwa_total /= len(test_loader)
dice_total /= len(test_loader)
IoU_total /= len(test_loader)

acc_UNet = [pwa_total, dice_total, IoU_total]

print("UNet")
print(f"Pixel-wise Accuracy: {pwa_total}")
print(f"Mean Dice Coefficient: {dice_total}")
print(f"Mean IoU: {IoU_total}")

# Get the number of trainable parameters
num_trainable_params = count_trainable_parameters(model)
print(f"Number of trainable parameters: {num_trainable_params}")

In [ ]:
# Regular Unet Benchmark
input_channels = 1
num_classes = 2
num_filters = [32, 64, 128, 256]

model = Unet(input_channels, num_classes, num_filters, dropout = 0.2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss = DiceLoss(alpha=2)

model_path = '/Users/austin/Documents/GitHub/GombNet/trained_models/BasicUNet256.pth'
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
model.eval()

pwa_total = 0
dice_total = 0
IoU_total = 0

# Calculate the accuracy
for i in range(len(test_loader)):
    test = test_loader.dataset[i][0].unsqueeze(0)
    gt = test_loader.dataset[i][1].numpy()  # Convert to numpy array
    
    with torch.no_grad():
        probability = model(test)
        prediction = torch.sigmoid(probability)  # Use torch.sigmoid instead of F.sigmoid (deprecated)
    
    probability = probability.squeeze().cpu().numpy()
    prediction = prediction.squeeze().cpu().numpy()

    threshold = threshold_otsu(prediction)
    prediction = (prediction > threshold).astype(float)
    
    # Calculate metrics for original and switched ground truths
    pwa_original = np.sum(prediction == gt) / np.prod(gt.shape)
    dice_original = dice_coefficient(prediction, gt)
    iou_original = iou(prediction, gt)
    
    # There is not combinatorial here
    pwa_total += pwa_original
    dice_total += dice_original
    IoU_total += iou_original

# Calculate the average for each metric
pwa_total /= len(test_loader)
dice_total /= len(test_loader)
IoU_total /= len(test_loader)

acc_UNet_gomb = [pwa_total, dice_total, IoU_total]

print("UNet w Gomb")
print(f"Pixel-wise Accuracy: {pwa_total}")
print(f"Mean Dice Coefficient: {dice_total}")
print(f"Mean IoU: {IoU_total}")

# Get the number of trainable parameters
num_trainable_params = count_trainable_parameters(model)
print(f"Number of trainable parameters: {num_trainable_params}")

In [ ]:
# GombNet
input_channels = 1
num_classes = 2
num_filters = [32, 64, 128, 256]

model = TwoLeggedUnet(input_channels, num_classes, num_filters, dropout = 0.2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss = GombinatorialLoss(group_size = num_classes//2, loss = 'Dice', epsilon=1e-6, class_weights = None, alpha=2)

model_path = '/Users/austin/Documents/GitHub/GombNet/trained_models/TwoLeggedGraphene256.pth'
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
model.eval()

pwa_total = 0
dice_total = 0
IoU_total = 0

# Calculate the accuracy
for i in range(len(test_loader)):
    test = test_loader.dataset[i][0].unsqueeze(0)
    gt = test_loader.dataset[i][1].numpy()  # Convert to numpy array
    
    # Switch ground truth layers
    gt_switched = np.flip(gt, axis=0)
    
    with torch.no_grad():
        probability = model(test)
        prediction = torch.sigmoid(probability)  # Use torch.sigmoid instead of F.sigmoid (deprecated)
    
    probability = probability.squeeze().cpu().numpy()
    prediction = prediction.squeeze().cpu().numpy()

    threshold = threshold_otsu(prediction)
    prediction = (prediction > threshold).astype(float)
    
    # Calculate metrics for original and switched ground truths
    pwa_original = np.sum(prediction == gt) / np.prod(gt.shape)
    pwa_switched = np.sum(prediction == gt_switched) / np.prod(gt_switched.shape)
    
    dice_original = dice_coefficient(prediction, gt)
    dice_switched = dice_coefficient(prediction, gt_switched)
    
    iou_original = iou(prediction, gt)
    iou_switched = iou(prediction, gt_switched)
    
    # Take the highest value for each metric
    pwa_total += max(pwa_original, pwa_switched)
    dice_total += max(dice_original, dice_switched)
    IoU_total += max(iou_original, iou_switched)

# Calculate the average for each metric
pwa_total /= len(test_loader)
dice_total /= len(test_loader)
IoU_total /= len(test_loader)

acc_GombNet = [pwa_total, dice_total, IoU_total]

print("GombNet")
print(f"Pixel-wise Accuracy: {pwa_total}")
print(f"Mean Dice Coefficient: {dice_total}")
print(f"Mean IoU: {IoU_total}")

# Get the number of trainable parameters
num_trainable_params = count_trainable_parameters(model)
print(f"Number of trainable parameters: {num_trainable_params}")

In [ ]:
colors = ['#1f77b4', 'green', '#d62728', 'purple', '#ff7f0e']

In [ ]:
colors = ['#1f77b4', 'green', '#d62728', 'purple', '#ff7f0e']

In [ ]:
metrics = ['Pixel-wise Accuracy', 'Mean Dice Coefficient', 'Mean IoU']
bar_width = 0.2
space_between_bars = 0.03

index = np.arange(len(metrics))
fig, ax = plt.subplots(figsize=(9, 5))

# Adjusting the positions for uniform spacing
positions = [-1.5 * (bar_width + space_between_bars), 
             -0.5 * (bar_width + space_between_bars), 
              0.5 * (bar_width + space_between_bars), 
              1.5 * (bar_width + space_between_bars)]

# Plot bars for Random
bar1 = ax.bar(index + positions[0], acc_random, bar_width, label='Random Guessing', color='#1f77b4')

# Plot bars for UNet
bar2 = ax.bar(index + positions[1], acc_UNet, bar_width, label='U-Net w Dice Loss', color='green')

# Plot bars for UNet with Gomb loss
bar3 = ax.bar(index + positions[2], acc_UNet_gomb, bar_width, label='U-Net w Gomb Loss', color='#d62728')

# Plot bars for GombNet
bar4 = ax.bar(index + positions[3], acc_GombNet, bar_width, label='Gomb-Net', color='#ff7f0e')

# Add labels, title, and legend
ax.set_ylabel('Values')
ax.set_xticks(index)
ax.set_xticklabels(metrics)
ax.legend()

# Save and show the plot
plt.savefig('GombNet_vs_UNet.png', dpi=300)
plt.show()